<a href="https://colab.research.google.com/github/zhaocaiQ/Git-Tutorial/blob/master/Copy_of_keyword_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터추출 쿼리

##pymysql설치

In [ ]:
pip install pymysql

     |████████████████████████████████| 43 kB 1.8 MB/s 


In [ ]:
import pymysql
#데이터베이스 연동
conn = pymysql.connect(host='tkt-db-ko.cnirlhwrm55r.ap-northeast-2.rds.amazonaws.com', port=3306, user='admin', password='tkt202202', db='topic_keywords_db', charset='utf8')
cursor = conn.cursor()

In [ ]:
from datetime import datetime
date = datetime.today().strftime("%Y-%m-%d")

In [ ]:
print(date)

2022-02-27


#TOP_keyword

##daily_top25

In [ ]:
cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date(now()) ORDER BY weight DESC LIMIT 25;")
result = cursor.fetchall()
print(result)

()


##weekly_top25

In [ ]:
cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
result = cursor.fetchall()
print(result)

(('우크라', 28), ('키예프', 24), ('러시아', 18), ('강풍', 14), ('최다', 12), ('푸틴', 11), ('산불', 11), ('방역', 10), ('제재', 9), ('항전', 9), ('역대', 9), ('수도', 9), ('사망자', 9), ('윤석열', 9), ('확진', 9), ('별세', 8), ('대선', 8), ('평화', 7), ('지성', 7), ('대통령', 7), ('시대', 7), ('러시아군', 6), ('장관', 6), ('직접', 6), ('정부', 6))


##monthly_top25

In [ ]:
cursor.execute("SELECT keyword, CAST(SUM(weight) AS SIGNED) FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 25;")
result = cursor.fetchall()
print(result)

(('우크라', 28), ('키예프', 24), ('러시아', 18), ('강풍', 14), ('최다', 12), ('푸틴', 11), ('산불', 11), ('방역', 10), ('제재', 9), ('항전', 9), ('역대', 9), ('수도', 9), ('사망자', 9), ('윤석열', 9), ('확진', 9), ('별세', 8), ('대선', 8), ('평화', 7), ('지성', 7), ('대통령', 7), ('시대', 7), ('러시아군', 6), ('장관', 6), ('직접', 6), ('정부', 6))


#chart_data

##daily_chart

In [ ]:
#오늘 top1~25출력
cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = curdate() ORDER BY weight DESC LIMIT 25;")
today = dict(cursor.fetchall())
#전날 top1~25출력
cursor.execute("SELECT keyword, weight FROM keyword WHERE c_date = date_add(curdate(), interval -1 day) ORDER BY weight DESC LIMIT 25;")
yesterday = dict(cursor.fetchall())
daily_chart = []
for i in today.keys():
    if i in yesterday.keys():
        if (today[i] - yesterday[i]) > yesterday[i]:
            daily_chart.append([i, today[i], "up"])
        elif (today[i] - yesterday[i]) == yesterday[i]:
            daily_chart.append([i, today[i], "same"])
        else:
            daily_chart.append([i, today[i], "down"])
    else:
        daily_chart.append([i, today[i], "new"])

dic_data = {
    'data': daily_chart
}
print(dic_data)

{'data': []}


##weekly_chart

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = cursor.fetchall()
weekly_chart = {}
for i in range(5):
    cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) AND keyword = '{keywords[i][0]}' ORDER BY c_date;")
    daily_weight = list(cursor.fetchall())
    weekly_chart[keywords[i][0]] = [n[0] for n in daily_weight]

dic_data = {
    'data': weekly_chart
}
print(dic_data)

{'data': {'우크라': [28], '키예프': [24], '러시아': [18], '강풍': [14], '최다': [12]}}


##monthly_chart

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = cursor.fetchall()
monthly_chart = {}
for i in range(5):
    cursor.execute(f"SELECT weight FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) AND keyword = '{keywords[i][0]}' ORDER BY c_date;")
    daily_weight = list(cursor.fetchall())
    monthly_chart[keywords[i][0]] = [n[0] for n in daily_weight]

dic_data = {
    'data': monthly_chart
}
print(dic_data)

{'data': {'우크라': [28], '키예프': [24], '러시아': [18], '강풍': [14], '최다': [12]}}


#Twitter_data

##daily_twitter

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE c_date = date(now()) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE published = curdate() GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
twitter = cursor.fetchall()

daily_twitter = []
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for key in twitter:
        if word[0] in key[0]:
            daily_twitter.append([no, list(key)])
    no += 1
print(daily_twitter)

[]


##weekly_twitter

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE published >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND published <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
twitter = cursor.fetchall()

weekly_twitter = []
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for key in twitter:
        if word[0] in key[0]:
            weekly_twitter.append([no, list(key)])
    no += 1
print(weekly_twitter)

[[1, ['22-02-26_우크라', 43346, 43, 'SBS뉴스,우크라이나, 러시아, 정전협상, 평화협상, 회담']], [2, ['22-02-26_키예프', 139563, 6, 'SBS뉴스']], [3, ['22-02-26_러시아', 233142, 150, 'PrayForUkraine,SBS뉴스']], [4, ['22-02-26_강풍', 108341, 22, 'SBS뉴스,날씨, 오늘날씨,뉴스, 최신뉴스,연예,철도기상']], [5, ['22-02-26_최다', 124626, 10, 'rt,남자명품추천, 남자벨트추천, 여자가방추천,남자벨트추천, 샤넬가방, 셀린느가방,명품백, 여성가방, 미니백, 샤넬백,문재인케어, 문재인케어,에스펄몰, 샤넬,임영웅, LimYoungWoong, 임영웅사랑은늘도망가, 임영웅유튜브, 임영웅영웅시대,임영웅, LimYoungWoong, 임영웅유튜브, 임영웅_브평UP,임영웅, limyoungwoong, 임영웅유튜브, 임영웅_음원강자, 임영웅_독보적, 임영웅_뮤빗,임영웅, LimYoungWoong, 임영웅유튜브, 임영웅뮤빗어워즈,트와이스, TWICE,펜디미니바게트, 디올몽테인백']]]


##monthly_twitter

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) GROUP BY keyword ORDER BY SUM(weight) DESC LIMIT 5;")
keywords = list(cursor.fetchall())
cursor.execute("SELECT keyword_id, CAST(SUM(retweet) AS SIGNED), CAST(SUM(like_count) AS SIGNED), group_concat(DISTINCT(nullif(tags, 'empty'))) FROM twitter WHERE published >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND published <= LAST_DAY(NOW()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
twitter = cursor.fetchall()

monthly_twitter = []
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for key in twitter:
        if word[0] in key[0]:
            monthly_twitter.append([no, list(key)])
    no += 1
print(monthly_twitter)

[[1, ['22-02-26_우크라', 43346, 43, 'SBS뉴스,우크라이나, 러시아, 정전협상, 평화협상, 회담']], [2, ['22-02-26_키예프', 139563, 6, 'SBS뉴스']], [3, ['22-02-26_러시아', 233142, 150, 'PrayForUkraine,SBS뉴스']], [4, ['22-02-26_강풍', 108341, 22, 'SBS뉴스,날씨, 오늘날씨,뉴스, 최신뉴스,연예,철도기상']], [5, ['22-02-26_최다', 124626, 10, 'rt,남자명품추천, 남자벨트추천, 여자가방추천,남자벨트추천, 샤넬가방, 셀린느가방,명품백, 여성가방, 미니백, 샤넬백,문재인케어, 문재인케어,에스펄몰, 샤넬,임영웅, LimYoungWoong, 임영웅사랑은늘도망가, 임영웅유튜브, 임영웅영웅시대,임영웅, LimYoungWoong, 임영웅유튜브, 임영웅_브평UP,임영웅, limyoungwoong, 임영웅유튜브, 임영웅_음원강자, 임영웅_독보적, 임영웅_뮤빗,임영웅, LimYoungWoong, 임영웅유튜브, 임영웅뮤빗어워즈,트와이스, TWICE,펜디미니바게트, 디올몽테인백']]]


#Youtube_data

##daily_youtube

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE c_date = date(now()) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE published = curdate() GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
twitter = cursor.fetchall()

daily_youtube = []
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for key in twitter:
        if word[0] in key[0]:
            daily_youtube.append([no, list(key)])
    no += 1
print(daily_youtube)

[]


##weekly_youtube

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND DATE(c_date) <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE published >= ADDDATE(curdate(), - WEEKDAY(curdate())) AND published <= ADDDATE(curdate(), - WEEKDAY(curdate())+ 6) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
twitter = cursor.fetchall()

weekly_youtube = []
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for key in twitter:
        if word[0] in key[0]:
            weekly_youtube.append([no, list(key)])
    no += 1
print(weekly_youtube)

[[1, ['22-02-26_우크라', 3986689, 33748, 19193, 'JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영, 러시아, 러시아군, 키예프, 우크라이나 침공, 러시아 침공, 푸틴, 바이든, 젤렌스키, 우크라이나 상황, No War Please, 우크라이나 필사 저항, 경비대, 도시 폭격, 러시아 군함 꺼져라,source:영상, type:방송, genre:국제, format:기타, person:김종균,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 우크라이나, 전쟁, 러시아,KBS, kbs news, kbs 뉴스, kbs뉴스, 국제, 군사작전, 뉴스, 뉴스_기타, 대통령, 돈바스, 러시아, 사적, 선포, 우크라이나, 움직임, 유지향, 임박, 침공, 푸틴,KBS, 뉴스, KBS뉴스라이']], [2, ['22-02-26_키예프', 14004086, 66218, 14307, 'MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 우크라이나, 키예프, 전쟁, 러시아, Kyiv, Ukraine, Livecam, Russia, Putin,source:영상, type:방송, genre:국제, format:기타, person:호준석,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 우크라이나, 러시아, 전쟁,러시아, 우크라, 함락,source:영상, type:방송, genre:국제, format:기타, person:이상순,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, 

##monthly_youtube

In [ ]:
cursor.execute("SELECT keyword FROM keyword WHERE DATE(c_date) >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND DATE(c_date) <= LAST_DAY(NOW()) ORDER BY weight DESC LIMIT 5;")
keywords = list(cursor.fetchall())
cursor.execute("SELECT keyword_id, CAST(SUM(view_count) AS SIGNED), CAST(SUM(like_count) AS SIGNED), CAST(SUM(comment_count) AS SIGNED), group_concat(tags) FROM youtube WHERE published >= DATE_SUB(curdate(), INTERVAL (DAY(curdate())-1) DAY) AND published <= LAST_DAY(NOW()) GROUP BY keyword_id ORDER BY COUNT(keyword_id) DESC;")
twitter = cursor.fetchall()

monthly_youtube = []
no = 1
while len(keywords) > 0:
    word = keywords.pop(0)
    for key in twitter:
        if word[0] in key[0]:
            monthly_youtube.append([no, list(key)])
    no += 1
print(monthly_youtube)

[[1, ['22-02-26_우크라', 3986689, 33748, 19193, 'JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영, 러시아, 러시아군, 키예프, 우크라이나 침공, 러시아 침공, 푸틴, 바이든, 젤렌스키, 우크라이나 상황, No War Please, 우크라이나 필사 저항, 경비대, 도시 폭격, 러시아 군함 꺼져라,source:영상, type:방송, genre:국제, format:기타, person:김종균,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, sohnsukhee, 제이티비씨, 제이티비씨 뉴스, ㅓ슟, 제이티비시, 제이티비시 뉴스, 안나경, 한민용, 오대영,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 우크라이나, 전쟁, 러시아,KBS, kbs news, kbs 뉴스, kbs뉴스, 국제, 군사작전, 뉴스, 뉴스_기타, 대통령, 돈바스, 러시아, 사적, 선포, 우크라이나, 움직임, 유지향, 임박, 침공, 푸틴,KBS, 뉴스, KBS뉴스라이']], [2, ['22-02-26_키예프', 14004086, 66218, 14307, 'MBC, MBC뉴스, 뉴스데스크, newsdesk, 뉴스투데이, newstoday, 8시뉴스, 아침뉴스, 뉴스, 정오뉴스, news, 우크라이나, 키예프, 전쟁, 러시아, Kyiv, Ukraine, Livecam, Russia, Putin,source:영상, type:방송, genre:국제, format:기타, person:호준석,연합뉴스, yonhapnews, 뉴스, 영상, 이슈, 디지털, news, 화제, 우크라이나, 러시아, 전쟁,러시아, 우크라, 함락,source:영상, type:방송, genre:국제, format:기타, person:이상순,JTBC, JTBC NEWS, 뉴스, 뉴스룸, 손석희, newsroom, 